In [1]:

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from cytominer_eval import evaluate
from pycytominer.cyto_utils import infer_cp_features
from cytominer_eval.transform import metric_melt

In [2]:
data_loc = '/Users/mbornhol/git/lincs-cell-painting/profiles/2016_04_01_a549_48hr_batch1/'

In [33]:
df_ls = []
for plate in os.listdir(data_loc):
    df_plate = pd.read_csv(os.path.join(data_loc, plate, f'{plate}_augmented.csv.gz'))
    df_ls.append(df_plate)
df_orig = pd.concat(df_ls)

In [88]:
df = df_orig.copy()

In [89]:
df.Metadata_moa = df.Metadata_moa.replace(np.nan, 'unknown', regex=True)

In [90]:
df.to_csv('full_level3.csv')

In [91]:
# 1. Only keep the dose 6 and dose 0 (DMSO)
dmso = df.query("Metadata_broad_sample == 'DMSO'")
df = df.query("Metadata_dose_recode == 6 & Metadata_moa != 'unknown'")

In [92]:
# 2. Get rid of all compounds that have unknown MOAs
df = pd.concat(
 [dmso, df], ignore_index=True
)

In [93]:
# 3. Drop all single Moas
ls = df.Metadata_moa.value_counts() != 1
keys = ls[ls].keys()
df = df[df["Metadata_moa"].isin(keys)]

In [94]:
df.to_csv('sub_level3.csv')

In [80]:
features = infer_cp_features(df)
def drop_feats(df_old, features_old, threshold):
    drop_features = []
    for feat in features_old:
        df_old[feat].astype(float)
        if (np.abs(df_old[feat]) > threshold).any() or df_old[feat].isna().any():
            drop_features.append(feat)
    df_out = df.drop(drop_features,  axis = "columns")
    print("dropped {} features".format(len(drop_features)))
    return df_out, drop_features
df, dropped_feats = drop_feats(df, features, 500)

dropped 36 features


In [87]:
df.to_csv('level3_featselected_500_nadropped.csv')

In [81]:
df.Metadata_moa.value_counts()


unknown                                                                                                                                                                       3264
phosphodiesterase inhibitor                                                                                                                                                    165
adrenergic receptor antagonist                                                                                                                                                 155
cyclooxygenase inhibitor                                                                                                                                                       132
histamine receptor antagonist                                                                                                                                                  123
                                                                                                         

In [82]:
df.Metadata_dose_recode.value_counts()


6    7326
1    3264
Name: Metadata_dose_recode, dtype: int64

In [83]:
df.head()

,Metadata_plate_map_name,Metadata_broad_sample,Metadata_mg_per_ml,Metadata_mmoles_per_liter,Metadata_solvent,Metadata_Plate,Metadata_Well,Metadata_pert_id,Metadata_pert_mfc_id,Metadata_pert_well,...,Nuclei_Texture_Variance_DNA_5_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_ER_20_0,Nuclei_Texture_Variance_ER_5_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_Mito_20_0,Nuclei_Texture_Variance_Mito_5_0,Nuclei_Texture_Variance_RNA_10_0,Nuclei_Texture_Variance_RNA_20_0,Nuclei_Texture_Variance_RNA_5_0
0,C-7161-01-LM6-027,DMSO,0.0,0.0,DMSO,SQ00015211,A01,NaN,NaN,A01,...,3.3092,1.7817,1.6356,1.8804,1.3283,1.2213,1.4497,2.6352,2.5141,2.6443
1,C-7161-01-LM6-027,DMSO,0.0,0.0,DMSO,SQ00015211,A02,NaN,NaN,A02,...,3.1899,1.7643,1.6399,1.8684,1.3931,1.2644,1.4969,2.6828,2.5727,2.6636
2,C-7161-01-LM6-027,DMSO,0.0,0.0,DMSO,SQ00015211,A03,NaN,NaN,A03,...,3.2723,1.7755,1.6156,1.9111,1.4134,1.2674,1.5254,2.6753,2.5601,2.6655
3,C-7161-01-LM6-027,DMSO,0.0,0.0,DMSO,SQ00015211,A04,NaN,NaN,A04,...,3.2232,1.7309,1.5815,1.8600,1.4268,1.2970,1.5449,2.7313,2.6180,2.7203
4,C-7161-01-LM6-027,DMSO,0.0,0.0,DMSO,SQ00015211,A05,NaN,NaN,A05,...,3.3201,1.8222,1.6668,1.9285,1.4788,1.3254,1.6039,2.7037,2.5771,2.7020


False